In [1]:
import pandas as pd
import sys
import time
import string
from preprocessor import (
    clean_word, stemmer, pipe, remove_punctuation,
    normalize_money, normalize_number, normalize_weekday,
    normalize_month, create_stemmer, create_stop_words_remover
)
from functools import reduce
import gensim

# Filter

In [2]:
is_real_sentence = lambda s: (
    len(s) > 10 and
    len(s.split()) > 10 and
    s[0] in string.ascii_letters  and 
    '|' not in s and 
    '=' not in s and 
    'html' not in s and 
    ':' not in s and
    '/' not in s
)

In [3]:
annotated_words = '''asing atas badan baru berat 
besar bidang bintang bisa buah 
bulan bunga cabang cerah coklat 
dalam dasar dunia halaman harapan 
jalan jam jaringan kabur kaki 
kali kepala ketat kulit kunci 
layar lebat lingkungan mata membawa 
memecahkan menangkap mendorong menerima mengandung 
mengejar mengeluarkan mengikat mengisi menjaga 
menurunkan menyusun nilai panas pembagian 
rapat sarung tengah tinggi '''.split()

In [4]:
exception_words = [w for w in list(map(stemmer.stem, annotated_words)) if w not in annotated_words]

# Raw Data

In [5]:
begin = time.perf_counter()
raw = open('../idwiki-latest-pages-articles-full.xml', 'r').readlines()
print('elapsed time:', time.perf_counter() - begin)
WIKI_LEN = len(raw)

elapsed time: 8.599584020000002


# List and Clean Sentences

In [7]:
preprocess_sentence = pipe(
    remove_punctuation,
    normalize_money,
    normalize_number,
    normalize_weekday,
    normalize_month,
    create_stemmer(annotated_words, exception_words),
    create_stop_words_remover(annotated_words, exception_words),
)

In [9]:
clean_sentences = []
begin = time.perf_counter()
max_time = 2400

for i in range(WIKI_LEN):
    
    sentences = []
    for s in raw[i].split('.'):
        sentences.append(s + '.')
    for s in sentences:
        if not is_real_sentence(s):
            continue
        s = s.replace('[[', '').replace(']]', '').replace('\n', '').replace("''", "").replace("'''", "").replace('&quot;', '')
        clean_sentences.append(preprocess_sentence(s))
    if i % (WIKI_LEN//5000) == 0:
        elapsed = int(time.perf_counter() - begin)
        sys.stdout.write("\rRaw read: {0:.2f} % | Instances collected: {1}| Time elapsed: {2} s | Time left: {3} s".format(
            i/WIKI_LEN*100, len(clean_sentences), elapsed, max_time - elapsed
        ))
        sys.stdout.flush()
        if elapsed > max_time:
            print()
            print('Time\'s up, raw lines read: ', i)
            break

Raw read: 100.00 % | Instances collected: 398438| Time elapsed: 1148 s | Time left: 1252 s

In [10]:
f = open('../wikipedia_clean_sentences_longer.txt', 'w')
for s in clean_sentences:
    f.write(s + '\n')
f.close()

# Load Processed Sentences

In [2]:
clean_sentences = open('../wikipedia_clean_sentences_longer.txt', 'r').readlines()

In [3]:
tokenized_clean_sentences = []
begin = time.perf_counter()
for i in range(len(clean_sentences)):
    tokenized_clean_sentences.append(clean_sentences[i].split())  
    if i % (1000) == 0:
        sys.stdout.write("\r{0:.2f} | Time elapsed: {1} s".format(
            i/len(clean_sentences), int(time.perf_counter() - begin)
        ))
        sys.stdout.flush()

1.00 | Time elapsed: 2 s

In [4]:
EMBEDDING_SIZE = 50

In [5]:
begin = time.perf_counter()
embedding_model = gensim.models.Word2Vec(tokenized_clean_sentences, window=5, size=EMBEDDING_SIZE, workers=7)
print('elapsed time:', time.perf_counter() - begin)

elapsed time: 16.03329802799999


In [6]:
embedding_model.wv.save_word2vec_format('../wikipedia_indonesia_embedding{}_more.model'.format(EMBEDDING_SIZE))

/home/nakama/Documents/StateOfTheArtWSD/nlp/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [7]:
embedding_model.wv.most_similar(positive='kompas')

[('tvku', 0.7483767867088318),
 ('tabloid', 0.7127206325531006),
 ('koran', 0.7001707553863525),
 ('mop', 0.6975473761558533),
 ('berita', 0.6966379880905151),
 ('tvri', 0.6819441914558411),
 ('tgl', 0.679807186126709),
 ('kamis', 0.6540974974632263),
 ('senin', 0.6495336294174194),
 ('sabtu', 0.6477915048599243)]